In [17]:
import numpy as np
import pandas as pd

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input, Conv2D, MaxPooling2D
from keras.optimizers import Adam, Adadelta
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications.nasnet import NASNetLarge
from keras.backend import sparse_categorical_crossentropy

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import os
from os import path, listdir
import gc

import cv2
# from PIL import Image


In [1]:
# Getting The Data
train_dset = pd.read_csv('./data/train.csv')
train_image_path = 'C:/Users/abtex/Desktop/100DaysOfMLCode/kaggleKernels/humpback whale/data/train'
test_image_path = 'C:/Users/abtex/Desktop/100DaysOfMLCode/kaggleKernels/humpback whale/data/test'

test_csv = pd.read_csv('./data/sample_submission.csv')


train_y = train_dset['Id'].values
labels = pd.unique(train_y)
train_y = pd.Series(train_y).astype('category').cat.codes
idx_to_label = dict(zip(np.unique(train_y),labels))

train_x = []
test_x  = []
train_imgs = train_dset['Image'].values

data_gen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False
)

img_dim = (128, 128)
num_channels = 3
num_classes = len(np.unique(train_y))


for img in train_imgs:
    image = cv2.imread(path.join(train_image_path, img))
    image = cv2.resize(image, img_dim)
    image = np.array(image)
    train_x.append(image)

for img in listdir(test_image_path):
    image = cv2.imread(path.join(test_image_path, img))
    image = cv2.resize(image, img_dim)
    image = np.array(image)
    test_x.append(image)

train_x = np.array(train_x)
test_x = np.array(test_x)

print(train_x.shape)
print(test_x.shape)



Using TensorFlow backend.


(25361, 128, 128, 3)
(7960, 128, 128, 3)


In [18]:
print(np.shape(train_y))

(25361, 5005)


In [19]:
# base_model = NASNetLarge(
#                     input_shape=(*img_dim, num_channels,),
#                     include_top=False,
#                     weights='imagenet',
#                     input_tensor=None,
#                     pooling=None,
#                 )

# for layer in base_model.layers:
#     layer.trainable = False


# x = base_model.output
inputs = Input(shape=(*img_dim, num_channels,))
x = Conv2D(filters=32, kernel_size=3, activation='tanh')(inputs)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=3, activation='tanh')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)
x = Conv2D(filters=128, kernel_size=3, activation='tanh')(x)
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=x)

model.summary()

model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(lr=1e-3))

# data_gen.fit(train_x)

reduceLR = ReduceLROnPlateau(
                            monitor='val_acc', 
                            patience=2, 
                            verbose=1, 
                            factor=0.2, 
                            min_lr=0.00001
                        )

earlystop = EarlyStopping(monitor='acc', min_delta=0, patience=1, verbose=0, mode='auto', baseline=None)

tensorboard = TensorBoard(log_dir='./logs', write_images=True, batch_size=128, write_graph=True, write_grads=True)

checkpoint = ModelCheckpoint('./checkpoints/', monitor='val_loss', save_best_only=True)

# model.fit_generator(
# 					data_gen.flow(train_x, train_y, batch_size=128),
# 					shuffle=True, 
# 					epochs=15,
# 					callbacks=[earlystop, tensorboard, reduceLR, checkpoint]
# 				)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 126, 126, 32)      128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 124, 124, 128)     36992     
_________________________________________________________________
batch_normalization_5 (Batch (None, 124, 124, 128)     512       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 60, 60, 128)       147584    
__________

In [20]:
model.fit(train_x, train_y, batch_size=128, epochs=10, callbacks=[earlystop, tensorboard, reduceLR, checkpoint])

ResourceExhaustedError: OOM when allocating tensor with shape[460800,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_3/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=4331415, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_3/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_3/random_uniform/RandomUniform', defined at:
  File "c:\users\abtex\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\abtex\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\abtex\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_ins

In [ ]:
def topK(predictions, k = 5):
    predictions = [np.absolute(np.argsort(-1*x))[:k] for x in predictions]
    predictions = [idx_to_label[i] for prediction in predictions for i in prediction]
    return predictions

preds = model.predict(test_x)
prediction = topK(preds, 5)

print(preds[1])
print(prediction[1])

In [ ]:

predictions = pd.DataFrame(data=test_csv)
predictions.drop(columns=['Id'], inplace=True)
predictions['Id'] = prediction.join(' ')
predictions.to_csv('submission.csv', index=False)